<a href="https://colab.research.google.com/github/PashaIanko/Kaggle.RwandaCO2Emissions/blob/main/3_model_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
# Datasets
import pandas as pd
# Numerics
import numpy as np
# Plotting
import matplotlib.pyplot as plt
# Preprocessing & pipelines
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# Data management
from sklearn.model_selection import train_test_split
# Other
import os

pd.set_option('display.max_columns', None)

In [2]:
%%capture
GIT_DOWNLOAD_PATH = 'https://raw.githubusercontent.com/PashaIanko/Sklearn-Utils/main/'
FILES_LIST = [
    'path_manager.py',
    'sklearn_transformers.py',
    'sklearn_utils.py',
    'model.py'
]
GDRIVE_PATH = '/content/gdrive/MyDrive/'
PREPROC_TRIAL = 1
MODELS_TRIAL = 1
COMPETITION_PATH = GDRIVE_PATH + 'ML/Competitions/8.CO2Emissions/'
# --------------------------------------------------
try:
    from nbpep8.nbpep8 import pep8
except ModuleNotFoundError:
    !pip install pycodestyle
    !pip install --index-url https://test.pypi.org/simple/ nbpep8
from nbpep8.nbpep8 import pep8
# ---------------------------------------
def download_files(url_dict):
    for file, url in url_dict.items():
        print(f'Downloading {file}')
        !wget -O {file} {url} {file}
url_dict = {file: GIT_DOWNLOAD_PATH + file for file in FILES_LIST}
print('a')
download_files(url_dict)
# ---------------------------------------
import importlib
import path_manager
import sklearn_utils
import sklearn_transformers
import model
def reload_all(modules_list_):
    for module in modules_list_:
        importlib.reload(module)
MODULES_LIST = [
    path_manager,
    sklearn_utils,
    sklearn_transformers,
    model
]
reload_all(MODULES_LIST)
# ---------------------------------------
from path_manager import PathManager
from model import Model
from sklearn_utils import nan_statistics
from sklearn_utils import boxplot_regression
from sklearn_utils import get_correlated_attributes
from sklearn_utils import visualize_datasets_distributions
from sklearn_transformers import ColumnDropper
from sklearn_transformers import LogTransformer
# ---------------------------------------
from google.colab import drive
drive.mount('/content/gdrive')
manager = PathManager(
    competition_path=COMPETITION_PATH,
    preprocessing_trial=PREPROC_TRIAL,
    models_trial=MODELS_TRIAL
)
manager.setup_paths()

# Download the data

In [3]:
df_train = pd.read_csv(f'{manager.data_trial_path}/train_processed.csv')
df_val = pd.read_csv(f'{manager.data_trial_path}/val_processed.csv')
df_test = pd.read_csv(f'{manager.data_trial_path}/test_processed.csv')
df_submission = pd.read_csv(f'{manager.data_trial_path}/submission_processed.csv')

# Sample the subset

In [4]:
SUBSET_SIZE = 0.05
RANDOM_STATE = 42


df_subset, _ = train_test_split(
    df_train,
    train_size=SUBSET_SIZE,
    random_state=RANDOM_STATE
)

print(df_subset.shape)

(2531, 85)


# Find the baseline model

## Random forest regressor

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


grid = {
    'n_estimators': [300]
}

res = GridSearchCV(
    RandomForestRegressor(),
    grid,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    refit=True,
    cv=3,
    verbose=4,
    return_train_score=True
)

In [12]:
cv_results = res.fit(
    X=df_subset.iloc[:, :-1].values,
    y=df_subset['target']
)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


In [15]:
cv_results.cv_results_

{'mean_fit_time': array([33.1655151]),
 'std_fit_time': array([2.11504644]),
 'mean_score_time': array([0.07895931]),
 'std_score_time': array([0.00652069]),
 'param_n_estimators': masked_array(data=[300],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 300}],
 'split0_test_score': array([-1512.79940571]),
 'split1_test_score': array([-1661.62874604]),
 'split2_test_score': array([-1666.09800466]),
 'mean_test_score': array([-1613.5087188]),
 'std_test_score': array([71.23560849]),
 'rank_test_score': array([1], dtype=int32),
 'split0_train_score': array([-232.02623107]),
 'split1_train_score': array([-235.47486798]),
 'split2_train_score': array([-222.52059878]),
 'mean_train_score': array([-230.00723261]),
 'std_train_score': array([5.4778669])}

In [9]:
from sklearn.metrics import mean_squared_error

mean_squared_error(
    res.predict(
        df_subset.iloc[:, :-1]
    ),
    df_subset['target']
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


176.7344697906446

In [11]:
mean_squared_error(
    res.predict(
        df_val.iloc[:1500, :-1]
    ),
    df_val['target'][:1500]
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


1435.2928295346253

## K Neighbors regressor

In [13]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV


knn_esimator = GridSearchCV

ImportError: ignored